In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
#크롬 브라우저 열기
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='username'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'userpw'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)


In [3]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

In [5]:
def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
time.sleep(10)

In [6]:
tag_list =[]

def get_content(driver):
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    data = [content, tag_data]
    return data

In [7]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

In [8]:
word = "풀무원바른먹거리"
url = insta_searching(word)

driver.implicitly_wait(9)
driver.get(url)
time.sleep(9)

click_first(driver)

result_list=[]

target = 2
for i in range(target):
    try:
        data = get_content(driver)
        result_list.append(data)
        next_page(driver)
    except:
        time.sleep(9)
        next_page(driver)
    

In [15]:
df = pd.DataFrame(result_list)
df['?'] = df['?'].apply(lambda x : clean_text(x))
df = df.drop_duplicates(subset=['?'])
df.columns = ['본문','해쉬태그']
df

,본문,해쉬태그
0,.[풀무원- 바른 먹거리 교육] 3-5학년 엄마들이 좋아하는 교육. 🤗영양소의 종류...,"[0428, juneha, 풀무원바른먹거리]"
1,💕풀무원 바른 먹거리 온라인 비대면 교육💕1차시/2월17일2차시/2월24일풀무원 바...,"[풀무원, 풀무원바른먹거리, 바른먹거리교육, 비대면교육, 초등집콕생활]"


In [ ]:
#df.to_csv("인스타.csv", header = True, index = False, encoding='utf-8-sig')

In [16]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.131)
